## Context Encoder

Let's begin by importing tensorflow and the network


In [ ]:
import tensorflow as tf

from network.contextEncoder import ContextEncoderNetwork

Now we initialize the context encoder network and select the step we want to use for the reconstruction.

In [ ]:
sr = 16000
start_in_seconds = 0.1
side_length = 2048
gap_length = 1024
starting_sample_left_side = int(sr*start_in_seconds)
ending_sample_left_side = starting_sample_left_side + side_length
starting_sample_right_side = ending_sample_left_side + gap_length
ending_sample_right_side = starting_sample_right_side + side_length

best_step = 506000
batch_size = 256

In [ ]:
tf.reset_default_graph()

aContextEncoderNetwork = ContextEncoderNetwork(batch_size=batch_size, window_size=5120, gap_length=1024, 
                                             learning_rate=1e-5, name='test')

In [ ]:
pathToDatasetFolder = 'audio'
audios = StrechableNumpyArray()
i = 0
total = 0
for file_name in os.listdir(pathToDatasetFolder):
    if file_name.endswith('.wav'):      
        audio, sr = librosa.load(pathToDatasetFolder + '/' + file_name, sr=None)
        
        if np.sum(np.absolute(audio[ending_sample_left_side:starting_sample_right_side])) < gap_length*1e-4: 
            print(file_name, "doesn't meet the minimum amplitude requirement")
            continue
        
        audios.append(audio)
        i+=1
        
        if i > 100:
            i -= 100
            total += 100
            print("100 plus!", total)
print("there were: ", total+i)

audios = audios.finalize()
audios = np.reshape(audios, (total+i, len(audio))


In [ ]:
left_side = audios[:, starting_sample_left_side:ending_sample_left_side]
right_side = audios[:, starting_sample_right_side:ending_sample_right_side]
sides = np.concatenate((left_side, right_side), axis=1)
original_gaps = audios[:, ending_sample_left_side:starting_sample_right_side]

In [ ]:
batch_count = 20
reconstructed = anAutoEncoderNetwork.reconstructAudio(sides/2+0.5, best_step, max_batchs=batch_count)

In [ ]:
left_side = audios[:len(reconstructed), :ending_sample_left_side]
right_side = audios[:len(reconstructed), starting_sample_right_side:]
reconstructed_signals = np.concatenate((left_side, (reconstructed-0.5)*2, right_side), axis=1)
zeroed_signals = np.concatenate((left_side, (reconstructed)*0, right_side), axis=1)
reconstructed_original = np.concatenate((left_side, original_gaps[:len(reconstructed)], right_side), axis=1)

In [ ]:
"""Write files to disk"""
maximum = 256

maxv = np.iinfo(np.int16).max
for index in range(min(len(reconstructed_signals), maximum)):
    librosa.output.write_wav("recs/original_" + str(index) + ".wav", (audios[index] * maxv).astype(np.int16), sr)
    librosa.output.write_wav("recs/reconstructed_" + str(index) + ".wav", (reconstructed_signals[index] * maxv).astype(np.int16), sr)


In [ ]:
reconstructed_signal_to_evaluate = 85

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 7)
f, axarr = plt.subplots(1, 3, sharey='row')
stop_value = min(1, len(original_gaps)+1)

difference = original_gaps[reconstructed_signal_to_evaluate]-((reconstructed[reconstructed_signal_to_evaluate]-0.5)*2)

axarr[0].plot(original_gaps[reconstructed_signal_to_evaluate])
axarr[1].plot((reconstructed[reconstructed_signal_to_evaluate]-0.5)*2)
axarr[2].plot(difference)

print(np.sum(np.absolute(original_gaps[reconstructed_signal_to_evaluate])))
print(np.absolute(difference).sum())
print(np.linalg.norm(difference))
print(_pavlovs_SNR( original_gaps[reconstructed_signal_to_evaluate], (reconstructed[reconstructed_signal_to_evaluate]-0.5)*2))

In [ ]:
IPython.display.Audio(data=reconstructed_signals[reconstructed_signal_to_evaluate], rate=16000)
# IPython.display.Audio(data=zeroed_signals[reconstructed_signal_to_evaluate], rate=16000)
# IPython.display.Audio(data=audios[reconstructed_signal_to_evaluate], rate=16000)


In [ ]:
plt.plot(reconstructed_signals[reconstructed_signal_to_evaluate][starting_sample_left_side:ending_sample_right_side])

In [ ]:
def _pavlovs_SNR(y_orig, y_inp):
    norm_y_orig = np.linalg.norm(y_orig) + 1e-10
    norm_y_orig_minus_y_inp = np.linalg.norm(y_orig - y_inp)
    return 10 * np.log10((abs(norm_y_orig ** 2)) / abs((norm_y_orig_minus_y_inp ** 2)))

def _euclideanNorm(vector):
        squared = np.square(vector)
        summed = np.sum(squared, axis=1)
        return np.sqrt(summed + 1e-10)
    
fake_a = (reconstructed - 0.5) * 2
gap = original_gaps[:int(batch_count*batch_size)]

SNRs = np.zeros((len(fake_a),))
for index, signal in enumerate(fake_a):
    SNRs[index] = _pavlovs_SNR(gap[index], fake_a[index])

norm_orig = _euclideanNorm(gap)
error = (gap - fake_a)
reconstruction_loss = 0.5 * np.sum(np.square(error), axis=1) * (1 + 1 / norm_orig)


In [ ]:
np.where(SNRs>20)

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

sorted_SNR = sorted(SNRs)

import scipy.stats as stats
fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    


In [ ]:
sorted_SNR = sorted(reconstruction_loss)

import scipy.stats as stats
fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    

In [ ]:
len(np.where(SNRs<-1.2)[0])

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

import scipy.stats as stats
sorted_SNR = sorted(SNRs)

fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    


In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

import scipy.stats as stats
sorted_SNR = sorted(SNRs)

fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    
362000

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

import scipy.stats as stats
sorted_SNR = sorted(SNRs)

fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    


In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)

import scipy.stats as stats
sorted_SNR = sorted(SNRs)

fit = stats.norm.pdf(sorted_SNR, np.mean(sorted_SNR), np.std(sorted_SNR))  #this is a fitting indeed

plt.plot(sorted_SNR,fit,'-o')

plt.hist(sorted_SNR, 50, normed=True)    


In [ ]:
plt.scatter(SNRs, reconstruction_loss)

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 28)

examples = np.where((SNRs<-10) & (reconstruction_loss<5))[0]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index is not 0 and index%8 is 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 


In [ ]:
len( np.where((SNRs<-10) & (reconstruction_loss<5))[0])

In [ ]:
examples = np.where((SNRs<1) & (SNRs>-1))[0][:256]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index%8 == 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 
    

In [ ]:
len(np.where(SNRs>28)[0])

In [ ]:
examples = np.where((SNRs>28))[0][:256]

f, axarr = plt.subplots(8, 2, sharey='row')

for index, example in enumerate(examples):   
    if index%8 == 0:
        f, axarr = plt.subplots(8, 2, sharey='row')
    axarr[index%8, 0].plot(out_gaps[example])
    axarr[index%8, 1].plot(reconstructed[example]) 
    